# Digital Twin Fairways
WIP to use the OpenTNSim engine

In [1]:
import sys
sys.path.append(r"D:\01. Projecten\[130878] DTV vaarwegen\digitaltwin-waterway\dtv_backend")
sys.path.append(r"D:\02. Tools\OpenTNSim")

import datetime

import logging
import geojson
import simpy
import time
import json
import shapely
import pandas as pd
import networkx as nx
from networkx.readwrite import json_graph

# opentnsim import
from opentnsim import core

# library to load the fairway information network
import dtv_backend.fis
# the simpy processes and objects
import dtv_backend.simple
import dtv_backend.network
import dtv_backend.network.network_utilities
import dtv_backend.berthing
import dtv_backend.simulate
import dtv_backend.postprocessing

# reload for debugging purposes
%load_ext autoreload
%autoreload 2

### Input
You can define your input in a json configuration file. The relevant parts are sites, fleet and climate.

In [2]:
# example input
with open('config.geojson') as f:
    config = geojson.load(f)

### Fixed content

In [14]:
class CanWork(dtv_backend.berthing.CanBerth):
    """WIP"""
    
    def __init__(self, *args, **kwargs):
        """Initialize"""
        #print(super().__init__)
        super().__init__(*args, **kwargs)
    
    def work_for(self, operator, with_berth=False):
        """Work for an operator by listening to tasks"""
        while True:
            # Get event for message pipe
            task = yield operator.get_task()
            # Where to get the cargo
            source = task.get("source")
            # Where to take it
            destination = task.get("destination")
            # How much
            max_load = task.get("max_load")

            # TODO: consider notifying the operator on progress
            # Notify operator of load changes so extra tasks can be planned
            # operator.send_message() or something...
            yield from self.load_move_unload(
                source, destination, max_load, with_berth=with_berth
            )
    
    def load_move_unload(self, source, destination, max_load=None, with_berth=False):
        """do a full A to B cycle"""
        with self.log_context(message="Cycle", description="Load move unload cycle"):
            # Don't sail to empty source
            if source.container.level <= 0:
                return

            if not with_berth:
                yield from self.move_to(source)
            else:
                yield from self.move_to_with_berth(source)

            # determine what the cargo to take
            cargo_for_trip = self.container.capacity
            yield from self.load_at(source, cargo_for_trip)

            # Don't sail empty
            if self.container.level <= 0:
                return

            if not with_berth:
                yield from self.move_to(destination)
            else:
                yield from self.move_to_with_berth(destination)
            yield from self.unload_at(destination)

In [15]:
Port = type('Port', (core.Identifiable, core.HasResource, core.HasContainer, core.ExtraMetadata), {})

In [16]:
Ship = type('Ship', (CanWork, dtv_backend.berthing.CanBerth, core.Identifiable, core.HasContainer, core.Movable, core.Locatable, core.ExtraMetadata), {})

### What is to be v3_run

In [17]:
# always start at now
now = datetime.datetime.now()
initial_time = now.timestamp()
env = simpy.Environment(initial_time=initial_time)
env.epoch = now

# default no berth
with_berth = config.get("options", {}).get("with_berth", False)

# read the network from google for performance reasons
url = "https://zenodo.org/record/6673604/files/network_digital_twin_v0.3.pickle?download=1"
G = dtv_backend.fis.load_fis_network(url)
env.FG = G

In [18]:
# ports
#logger.info("Loading ports ⚓")
ports = []
for site in config["sites"]:
    #port = dtv_backend.simple.Port(env, **site["properties"], **site)
    port  = Port(env=env, **site["properties"], **site)
    ports.append(port)

In [19]:
site['properties']

{'Wkt': 'POINT (4.9348192175000651 52.0388091591562372)',
 'X': 4.934819217500065,
 'Y': 52.03880915915624,
 'n': 'B21778_B',
 'name': 'B21778_B',
 'cargoType': 'Dry Bulk',
 'capacity': 10000,
 'level': 10000,
 'loadingRate': 198,
 'loadingRateVariation': 102}

In [20]:
# ships
#logger.info("Loading ships 🚢")
ships = []
for ship in config["fleet"]:
    kwargs = {}
    kwargs.update(ship)
    kwargs.update(ship["properties"])
    kwargs['v'] = 3
    kwargs['route'] = [feature['properties']['n'] for feature in config['route']]
    # the ship needs to know about the climate
    if "climate" in config:
        kwargs["climate"] = config["climate"]
    #ship = dtv_backend.simple.Ship(env, **kwargs)
    ship = Ship(env=env, **kwargs)
    ships.append(ship)

In [21]:
route = [feature['properties']['n'] for feature in config['route']]

In [22]:
ship = ships[0]
vars(ship)


{'metadata': {'type': 'Feature',
  'properties': {'CEMT-class': 'Va',
   'RWS-class': 'M8',
   'Vessel type': 'Motorvessel',
   'Description (English)': 'Large Rhine vessel',
   'Description (Dutch)': 'Groot Rijnschip',
   'Beam [m]': 11.4,
   'Beam 10% percentile [m]': 10.6,
   'Beam 50% percentile [m]': 11.45,
   'Beam 90% percentile [m]': 11.45,
   'Length [m]': 110,
   'Length 10% percentile [m]': 93,
   'Length 50% percentile [m]': 110,
   'Length 90% percentile [m]': 110,
   'Draught loaded [m]': 3.5,
   'Draught average [m]': 3.14,
   'Draught empty [m]': 1.4,
   'Height average [m]': 6.21,
   'Load Weight average [ton]': 2286,
   'Load weight maximum [ton]': 2689,
   'speed': 3,
   'image': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQRvKRniAxUXUWzmByw7CRFYD5fTqOtFTDVkw&usqp=CAU',
   'capacity': 2286,
   'name': 'Schip',
   'count': 4},
  'CEMT-class': 'Va',
  'RWS-class': 'M8',
  'Vessel type': 'Motorvessel',
  'Description (English)': 'Large Rhine vessel',
  'Descr

In [23]:
# operator
#logger.info("Loadig operator 👩‍💼")
# Setup and start the simulation
operator = dtv_backend.simple.Operator(
    env=env, ships=ships, **config["operator"])
# The ships do work for the operator
for ship in ships:
    env.process(ship.work_for(operator, with_berth=with_berth))
# The opertor plans the work move everything from A to B
env.process(operator.plan(ports[0], ports[1]))

<Process(plan) object at 0x19140fe3190>

In [24]:
#logger.info("Running simulation 👩‍💻")
# Run for n days
n_days_in_future = now + datetime.timedelta(days=60)
env.run(until=n_days_in_future.timestamp())

AttributeError: 'Ship' object has no attribute 'move_to'

In [ ]:
# to be returned by function v3_run
{
    "env": env,
    "operator": operator,
    "ships": ships,
    "config": config,
    "ports": port,
}

### WIP snippets